# Apache Airflow

## Intro

Apache Airflow (or simply _Airflow_) is a platform to programmatically 
- Create (Author)
- Schedule
- Monitor 

workflows.

We can use airflow to author workflows as directed acyclic graphs (DAGs) of tasks. The airflow scheduler executes the tasks on an array of workers while following the specified dependencies. Rich command line utilities make performing complex surgeries on DAGs a snap. The rich user interface makes it easy to visualize pipelines running in production, monitor progress, and troubleshoot issues when needed.

When workflows are defined as code, they become more maintainable, versionable, testable, and collaborative.

```bash
# airflow needs a home, ~/airflow is the default,
# but you can lay foundation somewhere else if you prefer
# (optional)
export AIRFLOW_HOME=~/airflow

# install from pypi using pip
pip install apache-airflow

# initialize the database
airflow db init

airflow users create \
    --username admin \
    --firstname Peter \
    --lastname Parker \
    --role Admin \
    --email spiderman@superhero.org

# start the web server, default port is 8080
airflow webserver --port 8080

# start the scheduler
# open a new terminal or else run webserver with ``-D`` option to run it as a daemon
airflow scheduler

# visit localhost:8080 in the browser and use the admin account you just
# created to login. Enable the example_bash_operator dag in the home page
```

In [1]:
# Command line tool, subcommands
!airflow -h

usage: airflow [-h] GROUP_OR_COMMAND ...

positional arguments:
  GROUP_OR_COMMAND

    Groups:
      celery         Celery components
      config         View configuration
      connections    Manage connections
      dags           Manage DAGs
      db             Database operations
      kubernetes     Tools to help run the KubernetesExecutor
      pools          Manage pools
      providers      Display providers
      roles          Manage roles
      tasks          Manage tasks
      users          Manage users
      variables      Manage variables

    Commands:
      cheat-sheet    Display cheat sheet
      info           Show information about current Airflow and environment
      kerberos       Start a kerberos ticket renewer
      plugins        Dump information about loaded plugins
      rotate-fernet-key
                     Rotate encrypted connection credentials and variables
      scheduler      Start a scheduler instance
      sync-perm      Update permissions for e

### Running a Workflow

Running a simle task
```bash
Airflow run <dag_id> <task_id> <start_date>
```

Example
```bash
# Run a task instance
airflow tasks run example_bash_operator runme_0 2021-01-01
# Run a backfill over 2 days
airflow dags backfill example_bash_operator \
    --start-date 2021-01-01 \
    --end-date 2021-01-02
```

### DAG

> In Airflow, a pipeline is represented as a Directed Acyclic Graph or DAG. The nodes of the graph represent tasks that are executed. The directed connections between nodes represent dependencies between the tasks. Representing a data pipeline as a DAG makes much sense, as some tasks need to finish before others can start. [source](https://datacamp.com)



In [ ]:
from airflow.models import DAG
from datetime import datetime

# Default arguments (optional) that will be applied to the components of DAG
default_args = {
    'owner': 'name',
    'email': 'name@gmail.com',  # for alerting
    'start_date': datetime(2020, 1, 20),
    'retries': 2
}

etl_dag = DAG('example_etl', default_args=default_args)

Airflow shell command can provide a lot of useful information when creating and troubleshooting workflows.

In [10]:
!airflow dags -h

usage: airflow dags [-h] COMMAND ...

Manage DAGs

positional arguments:
  COMMAND
    backfill      Run subsections of a DAG for a specified date range
    delete        Delete all DB records related to the specified DAG
    list          List all the DAGs
    list-jobs     List the jobs
    list-runs     List DAG runs given a DAG id
    next-execution
                  Get the next execution datetimes of a DAG
    pause         Pause a DAG
    report        Show DagBag loading report
    show          Displays DAG's tasks with their dependencies
    state         Get the status of a dag run
    test          Execute one single DagRun
    trigger       Trigger a DAG run
    unpause       Resume a paused DAG

optional arguments:
  -h, --help      show this help message and exit


### Airflow Webserver and UI

In [12]:
!airflow webserver -h

usage: airflow webserver [-h] [-A ACCESS_LOGFILE] [-L ACCESS_LOGFORMAT] [-D]
                         [-d] [-E ERROR_LOGFILE] [-H HOSTNAME] [-l LOG_FILE]
                         [--pid [PID]] [-p PORT] [--ssl-cert SSL_CERT]
                         [--ssl-key SSL_KEY] [--stderr STDERR]
                         [--stdout STDOUT] [-t WORKER_TIMEOUT]
                         [-k {sync,eventlet,gevent,tornado}] [-w WORKERS]

Start a Airflow webserver instance

optional arguments:
  -h, --help            show this help message and exit
  -A ACCESS_LOGFILE, --access-logfile ACCESS_LOGFILE
                        The logfile to store the webserver access log. Use '-' to print to stderr
  -L ACCESS_LOGFORMAT, --access-logformat ACCESS_LOGFORMAT
                        The access log format for gunicorn logs
  -D, --daemon          Daemonize instead of running in the foreground
  -d, --debug           Use the server that ships with Flask in debug mode
  -E ERROR_LOGFILE, --error-logfile ERROR_

In [13]:
# Start webserver on port 8051
!airflow webserver -p 8051

## Airflow DAGs: Operators, Tasks, Scheduling

### Operators

Airflow Operators represent a single task (e.g. running a command, python script, etc.) in a workflow. There are various operators that perform different tasks. A list of operators can be found [here](https://airflow.apache.org/docs/apache-airflow/stable/_api/airflow/operators/index.html). Some of those are:
- `DummyOperator`
- `BashOperator`
- `PythonOperator`
- `PostgresOperator`

Note that operators generally don't share information. 

#### `DummyOperator`
DummyOperator can be used for troubleshooting or testing.

```bash
DummyOperator(task_id='example', dag=dag)
```

#### `BashOperator`

- Allows to specify any given Shell command or script and add it to an Airflow workflow.
- It is possible to specify environment variables for the bash command.
- In Airflow, tilde character does not by default represent home directory. This can be fixed with env variables.

In [ ]:
from airflow.operators.bash_operator import BashOperator

# Execute a Command
task1 = BashOperator(    
    task_id='bash_example',  # Name that shows up in the UI
    bash_command='echo "Hello!"',
    dag=my_dag  # Add the task to the DAG
)

# Execute a Script
task2 = BashOperator(
    task_id='bash_script',
    bash_command='runupdate.sh',
    dag=my_dag)
    
task3 = BashOperator(
    task_id='clean_address',
    bash_command='cat address.txt | awk "NF==10" > cleaned.txt',
    dag=my_dag)

Define multiple `BashOperator`s within a workflow (same DAG):

In [ ]:
# Define the first operator
consolidate = BashOperator(
    task_id='consolidate_task',
    bash_command='consolidate_data.sh',
    dag=analytics_dag)

# Define the second operator
push_data = BashOperator(
    task_id='pushdata_task',
    bash_command='push_data.sh',
    dag=analytics_dag)

### Tasks

Tasks are instances of operators. Although tasks can be assigned to variables, a task is referred by its `task_id` instead of `variable name`.

Task Dependencies and Task Ordering:

Task dependencies are referred to as _upstream_ or _downstream_ tasks. Upstream tasks: are compleated prior to any downstream tasks. Since Airflow 1.8, dependencies are defined using the _bitshift_ operators.
- Upstream operator: `>>`  (before)
- Downstream operator: `<<`  (after)

In [ ]:
# Define the tasks
task1 = BashOperator(task_id='first_task',
                     bash_command='echo 1',
                     dag=example_dag)

task2 = BashOperator(task_id='second_task',
                     bash_command='echo 2',
                     dag=example_dag)

# Set first_task to run before second_task
task1 >> task2  # or task2 << task1


### Scheduling